In [16]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# load in suburb names
suburb_list = pd.read_csv("./actSuburbs.csv")
# valid_suburbs = []

sale_data = pd.DataFrame(columns=["suburb", "year", "type", "price"])

# use a headless browser (saves time)
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--headless")

# set up the browser
driver = webdriver.Chrome(options=chrome_options)

def find_suburb(suburb):
    print("finding data for", suburb)
    driver.get("https://www.allhomes.com.au/ah/research/property-and-past-sales")
    
    search_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "ss"))
    )
    
    # input suburb name
    search_field.clear()
    search_field.send_keys(suburb)
    
    # gather suggestions
    location_suggestions = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "ui-menu-item"))
    )
    
    # check suggestions, click correct on
    location = 0
    while True:
        text = location_suggestions[location].find_element_by_tag_name("div").get_attribute("innerText")
        icon = location_suggestions[location].find_element_by_tag_name("img").get_attribute("src")
        if (f"{suburb}," in text) and ("ACT" in text) and (icon[49:icon.find("?")] == "division-icon.png"):
            location_suggestions[location].click()
            check = True
            break
        else:
            location = location + 1
            if location == len(location_suggestions):
                check = False
                break
    
    if check:
        print("found", suburb)
        check_suburb(suburb)
        
    else:
        print("did not find", suburb)

def check_suburb(suburb):
    fields = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "four_column_wrapper"))
    )

    check = False    
    for field in fields:
        entries = field.find_elements_by_tag_name("a")

        for entry in entries:
            text = entry.get_attribute("innerText")
            if (suburb in text) and ("ACT" in text):
                check = True
                check_link = entry

    if check:
        print("found suburb link for", suburb)
        check_link.click()
        check_years(suburb)

    else:
        print("did not find suburb link for", suburb)

def check_years(suburb):
    try:
        year_links = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".sales-history-wrapper a.research-year-button"))
        )

        valid_year_links = []
        for year_link in year_links:
            text = year_link.get_attribute("innerText")
            if int(text) > 2010:
                valid_year_links.append([year_link.get_attribute("innerText"), year_link.get_attribute("href")])

        if len(valid_year_links) < 1:
            print("no valid years")

        else:
            print(len(valid_year_links), "valid years")
            for link in valid_year_links:
                print(suburb, link[0], link[1])
                scrape_year(suburb, link[0], link[1])

    except:
        print("no year links for", suburb)

def scrape_year(suburb, year, url):
    driver.get(url)
    
    sale_rows = WebDriverWait(driver, 10).until(
        EC.presence_all_all_elements_location((BY.CLASS_NAME, "research-full-details"))
    )
    
    for sale in sale_rows:
        price = sale.find_element_by_class_name("research-table-row").get_attribute("innerText")
        
    

for suburb in suburb_list.suburb:
    find_suburb(suburb)
    
driver.close()

finding data for Acton
found Acton
found suburb link for Acton
10 valid years
Acton 2012 https://www.allhomes.com.au/ah/research/acton/121451212/sale-history?year=2012
Acton 2013 https://www.allhomes.com.au/ah/research/acton/121451212/sale-history?year=2013
Acton 2014 https://www.allhomes.com.au/ah/research/acton/121451212/sale-history?year=2014
Acton 2015 https://www.allhomes.com.au/ah/research/acton/121451212/sale-history?year=2015
Acton 2016 https://www.allhomes.com.au/ah/research/acton/121451212/sale-history?year=2016
Acton 2017 https://www.allhomes.com.au/ah/research/acton/121451212/sale-history?year=2017
no year links for Acton
finding data for Ainslie


WebDriverException: Message: chrome not reachable
  (Session info: chrome=89.0.4389.90)
